In [1]:
import typing as ty
from pathlib import Path
from logzero import logger
import json

In [2]:
PATH_GMAP_API = './configurations/gmap_apikey'
PATH_DIR_LOAD_LIST_JSON = '/media/DATA/mitsuzaw/mmd-tst-variable-detector/demos/demo_sumo_monaco/output/mmd-algorithm-one/output_jsons'
PATH_SUMO_NET_XML = '/home/mitsuzaw/codes/dev/sumo-sim-monaco/simulation_extractions/sumo_configs/base/standard_config/in/most.net.xml'

In [3]:
# function to extract geo-coding system from SUMO net file.
# Must be in the library
path_sumo_net_xml = Path(PATH_SUMO_NET_XML)

import xml.etree.ElementTree as ET

geolocation_obj: ty.Dict = {}

iter_elem = ET.iterparse(path_sumo_net_xml.as_posix())
for _event, _elem in iter_elem:
    if _elem.tag == 'location':
        geolocation_obj = _elem.attrib
        geolocation_obj['netOffset'] = [float(v) for v in _elem.attrib['netOffset'].split(',')]
        geolocation_obj['convBoundary'] = [float(v) for v in _elem.attrib['convBoundary'].split(',')]
        geolocation_obj['origBoundary'] = [float(v) for v in _elem.attrib['origBoundary'].split(',')]
        break
# end for
geolocation_obj

# Note: the geo-coding is already epsg:4326. Hence no need to convert.

{'netOffset': [-368142.4, -4841896.61],
 'convBoundary': [0.0, 0.0, 9975.22, 6362.01],
 'origBoundary': [7.362913, 43.718985, 7.485778, 43.776857],
 'projParameter': '+proj=utm +zone=32 +ellps=WGS84 +datum=WGS84 +units=m +no_defs'}

In [4]:
try:
    import sumolib
except ImportError:
    !pip install sumolib

In [5]:
path_dir_load_list = Path(PATH_DIR_LOAD_LIST_JSON)
seq_list_road_list_json = list(sorted(path_dir_load_list.rglob('*.json')))
assert len(seq_list_road_list_json) > 0, f'No json files found in {path_dir_load_list}'

In [6]:
# json.load(seq_list_road_list_json[0].open())

In [7]:
from sumolib.net import Net
from sumolib.net.edge import Edge
from sumolib.net.node import Node
from sumolib.net.lane import Lane
import sumolib

In [8]:
# convert the coordinates. from: https://sumo.dlr.de/docs/Tools/Sumolib.html#coordinate_transformations
net = sumolib.net.readNet(PATH_SUMO_NET_XML)

lon, lat = net.convertXY2LonLat(3692.09, 462.98)

logger.info(f'{lat},{lon}')

[I 240204 14:40:54 3492308146:6] 43.72319396799985,7.40879887680717


In [9]:
assert Path(PATH_GMAP_API).exists(), f'File {PATH_GMAP_API} not found'
path_apikey = Path(PATH_GMAP_API)
_api_key = path_apikey.open().read()

In [43]:
# an exmaple to get the edges and lanes

def get_lane_id_dict(net: sumolib.net.Net) -> ty.Dict[str, Lane]:
    """Get the lane id to lane object dictionary from the net object.
    """
    seq_edge_obj = net.getEdges(withInternal=True)
    seq_lane_id2lane = {}
    for _edge_obj in seq_edge_obj:
        _seq_lane_obj = _edge_obj.getLanes()
        for _lane_obj in _seq_lane_obj:
            seq_lane_id2lane[_lane_obj.getID()] = _lane_obj
        # end for
    # end for
    logger.debug(f'Number of lanes: {len(seq_lane_id2lane)}')
    return seq_lane_id2lane


def get_edge_polygon_shape_wgs84(dict_lane_id2lane: ty.Dict[str, Lane], 
                                 lane_id: str,
                                 ) -> ty.Optional[ty.List[ty.Tuple[float, float]]]:
    """Get the polygon shape of the edge in WGS84 coordinate system.
    
    Returns
    -------
    ty.Optional[ty.List[ty.Tuple[float, float]]]
        The polygon shape of the edge in WGS84 coordinate system, (latitute, longitude)
    """
    if lane_id not in dict_lane_id2lane:
        logger.error(f'lane_id {lane_id} not found in the net.')
        return None
    # end if
    lane_obj = dict_lane_id2lane[lane_id]
    __polygon_shapes = lane_obj.getShape()
    if len(__polygon_shapes) == 0:
        logger.error(f'lane_id {lane_id} has no shape.')
        return None
    # end if
    __polygon_wsg84 = [net.convertXY2LonLat(__t_xy[0], __t_xy[1]) for __t_xy  in __polygon_shapes]
    logger.debug(f'Number of polygon shapes: {len(__polygon_wsg84)}')
    __seq_lat_lon = [(v[1], v[0]) for v in __polygon_wsg84]
    return __seq_lat_lon


In [11]:
# An example to get the position of a junction.
# note: ids starting with ':' are junctions. Hence, impossible to get it from the edge API.
# net.getEdge(':132185')

# sub-procedure to get positions of "junctions" from the net file.
# When ":" is the prefix of the edge id, it is a junction.

# target_junction_id = '141613'

def get_junction_object(net: sumolib.net.Net) -> ty.Dict[str, Node]:
    """Get the junction object from the net object.
    """
    seq_node_obj = net.getNodes()
    dict_nodeID2node= {__node_obj.getID(): __node_obj for __node_obj in seq_node_obj}
    return dict_nodeID2node


def get_junction_position_wgs84(dict_nodeID2node: ty.Dict[str, Node], 
                                junction_id: str,
                                ) -> ty.Optional[ty.Tuple[float, float]]:
    """Get the position of the junction in WGS84 coordinate system.
    
    Returns
    -------
    ty.Optional[ty.Tuple[float, float]]
        The position of the junction in WGS84 coordinate system.
    """
    if junction_id not in dict_nodeID2node:
        logger.error(f'junction_id {junction_id} not found in the net.')
        return None
    # end if
    __x, __y = dict_nodeID2node[junction_id].getCoord()
    lon, lat = net.convertXY2LonLat(__x, __y)
    logger.debug(f'{lat},{lon}')
    return lat, lon

Gmap API is: https://jupyter-gmaps.readthedocs.io/en/latest/api.html

In [62]:
import gmaps
from IPython.display import display


time_bucket_file = 4
weight_file_content = json.load(seq_list_road_list_json[time_bucket_file].open())

logger.info(f'N(weights) = {len(weight_file_content)}')

dict_nodeID2node = get_junction_object(net)
dict_laneID2lane = get_lane_id_dict(net)

# Use google maps api
gmaps.configure(api_key=_api_key) # Fill in with your API key
# # Create the map
# fig = gmaps.figure(center=(51.5, -0.1), zoom_level=12)
fig = gmaps.figure()


seq_ids_node = [t_score for t_score in weight_file_content if ':' in t_score[0]]
seq_ids_lanes = [t_score for t_score in weight_file_content if ':' not in t_score[0]]

# -------------------------------------
# adding junction pins to Google Maps
seq_positions = []
seq_position_label = []

for __t_node_id in seq_ids_node:
    __junction_id = __t_node_id[0].split('_')[0].strip(':')
    _t_position = get_junction_position_wgs84(dict_nodeID2node, __junction_id)
    if _t_position is None:
        continue
    # end if    
    seq_positions.append(_t_position)
    seq_position_label.append(f'id={__t_node_id[0]}, weights={__t_node_id[1]}')
# end for

# -------------------------------------
# Adding lane polygons to Google Maps
seq_polygon_obj = []

for __t_lane_id in seq_ids_lanes:
    __lane_id = __t_lane_id[0]
    _t_position = get_edge_polygon_shape_wgs84(dict_laneID2lane, __lane_id)
    if _t_position is None:
        continue
    # end if    
    _plygon_obj = gmaps.drawing.Polygon(_t_position, stroke_color='blue', fill_color='blue')
    # Adding polygon object
    seq_polygon_obj.append(_plygon_obj)
    # Adding marker at the polygon index 0
    seq_positions.append(_t_position[0])
    seq_position_label.append(f'id={__t_lane_id[0]}, weights={__t_lane_id[1]}')
# end for

# -------------------------------------
# fig.add_layer(drawing)
# creating the layer
markers = gmaps.marker_layer(seq_positions, hover_text=seq_position_label)
drawing = gmaps.drawing_layer(
    features=seq_polygon_obj,
    show_controls=False
)
# -------------------------------------
# Adding layer object
fig.add_layer(markers)
fig.add_layer(drawing)
display(fig)

[I 240204 15:35:55 668964303:8] N(weights) = 4
[D 240204 15:35:56 3952494268:14] Number of lanes: 8621


[D 240204 15:35:56 416692481:34] 43.735285373868926,7.419442499402342
[D 240204 15:35:56 416692481:34] 43.73307263235048,7.420069413044834
[D 240204 15:35:56 416692481:34] 43.72913336529539,7.410581779457323
[D 240204 15:35:56 3952494268:39] Number of polygon shapes: 10


Figure(layout=FigureLayout(height='420px'))

In [ ]:
from ipywidgets.embed import embed_minimal_html
import gmaps
gmaps.configure(api_key=_api_key)
embed_minimal_html('export.html', views=[fig])